In [ ]:
!pip install autokeras -q

## 8.3.1 Model scheduling with Hyperband

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import autokeras as ak
import keras_tuner as kt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def build_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(
  units=hp.Int('units', min_value=32, max_value=512, step=32),
  activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
  return model

tuner = kt.Hyperband(
  build_model,
  objective='val_loss',
  max_epochs=10,
  factor=3,
  hyperband_iterations=2,
  directory='result_dir',
  project_name='helloworld')

tuner.search(x_train, y_train,
  epochs=1,
  validation_data=(x_test, y_test))

Trial 21 Complete [00h 00m 14s]
val_loss: 0.3493499159812927

Best val_loss So Far: 0.2390340268611908
Total elapsed time: 00h 04m 40s
INFO:tensorflow:Oracle triggered exit


## 8.3.2 Faster convergence with pretrained weights in search space

In [ ]:
import tensorflow as tf
import autokeras as ak

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation()(output_node)
output_node = ak.ResNetBlock(pretrained=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
model = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=2, overwrite=True)
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test)

Trial 2 Complete [00h 00m 48s]
val_loss: 1.786433219909668

Best val_loss So Far: 1.7001031637191772
Total elapsed time: 00h 02m 16s
INFO:tensorflow:Oracle triggered exit
1563/1563 [==============================] - 27s 15ms/step - loss: 1.7725 - accuracy: 0.3676


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
313/313 [==============================] - 6s 16ms/step - loss: 1.7035 - accuracy: 0.3925


[1.7035101652145386, 0.39250001311302185]

In [ ]:
import tensorflow as tf
resnet = tf.keras.applications.ResNet50(
    include_top=False, 
    weights="imagenet")
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
import tensorflow as tf
import kerastuner as kt

def build_model(hp):
  if hp.Boolean('pretrained'):
    weights = "imagenet"
  else:
    weights = None
  resnet = tf.keras.applications.ResNet50(
      include_top=False, 
      weights=weights)
  if hp.Boolean('freeze'):
    resnet.trainable = False
  
  input_node = tf.keras.Input(shape=(32, 32, 3))
  output_node = resnet(input_node)
  output_node = tf.keras.layers.Dense(10, activation='softmax')(output_node)
  model = tf.keras.Model(inputs=input_node, outputs=output_node)
  model.compile(loss='sparse_categorical_crossentropy')
  return model

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

tuner = kt.RandomSearch( 
    build_model, 
    objective='val_loss', 
    max_trials=4,
    overwrite=True,
    directory='result_dir', 
    project_name='pretrained') 
      
tuner.search(x_train, y_train, 
             epochs=1, 
             validation_data=(x_test, y_test))

Trial 3 Complete [00h 02m 32s]
val_loss: 5.470385551452637

Best val_loss So Far: 3.4418113231658936
Total elapsed time: 00h 04m 45s
INFO:tensorflow:Oracle triggered exit
